In [ ]:
import time, torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import sys

def sys_print(ss):
    sys.stdout.write(ss)
    sys.stdout.flush()

from my_BM25 import BM25_searcher
elasticsearch_url="http://localhost:9205"
index_name="vhac_embed"
retriever = BM25_searcher(host=elasticsearch_url, index_name=index_name)


import pandas as pd
file_path = open('../data/test_path.txt').read()
df = pd.read_csv(file_path)

question_embedding_dict = torch.load('../data/embeddings.pt')
question_request_dict = torch.load('../data/quest_explained.pt')
answer_dict = dict()

for index, question in enumerate(df.question):
    embedding = question_embedding_dict[question]
    request = question_request_dict[question]
    docs = retriever.search_embed(query=question, embedding=embedding, top_k=16)
    # print(question)
    sys_print('.')

    for i in range(14):
        context = docs[i] + docs[i+1] + docs[i+2]
        prompt = f"Dựa vào thông tin >>> {context}\n\nHãy trả lời câu hỏi sau >>> {question}\nPhải đảm bảo các yêu cầu sau >>> {request}\nCâu trả lời của bạn là: "
